In [1]:
import pandas as pd

pd.set_option('display.max_rows', 10)

asset_classes = ['EQUITY', 'CASH']
countries = ["United Arab Emirates", "Turkey", "Saudi Arabia", "Czech Republic", "European Union", "Russian Federation", "United Kingdom", "Hungary", "Qatar", "Netherlands", "Kazakhstan", "Greece", "Kuwait", "PO", "TU", "United States", "Poland", "South Africa", "Romania"]
sectors = ["Financials", "Energy", "Cash", "Health Care", "Real Estate", "Consumer Discretionary", "Materials", "Consumer Staples", "Communication Services", "Industrials"]

with open("bemo_raw.txt", 'r') as file:
    bemo_raw = file.read()

rows = bemo_raw.split('\n')

# drop the header row
rows = rows[1:]

# get the date
val_date = rows[0][:10]

def delete_unnecessary_bits(s):

    s = s.replace('BARINGS EMERGING EMEA OPPORTUNITIES PLCKRYPTON', ' ')
    s = s.replace(val_date, ' ')
    
    for asset_class in asset_classes:
        s = s.replace(asset_class, ' ')

    for country in countries:
        s = s.replace(country, ' ')
    
    for sector in sectors:
        s = s.replace(sector, ' ')
    
    return s


for i in range(len(rows)):
    rows[i] = delete_unnecessary_bits(rows[i])

rows[1]

'   ADNOC DRILLING COMPANYBN12D39 2,225,680 AED 2.78%     '

In [2]:
# have to clean this up manually for now
with open("sedols_and_quantities_raw.txt", "w") as file:
    for line in rows:
        file.write(line + "\n")

In [3]:
with open("sedols_and_quantities.txt", 'r') as file:
    sedols_and_quantities = file.read()

sedols_and_quantities

'SEDOL,Quantity\n6545464,1030804\nBN12D39,2225680\nB03MN70,360828\nB403QG4,9987\nB12LZH9,320274\nB0LX3Y2,765014\nBMBQDF6,90590\nBZ1MXR7,1423484\n6761000,101\nB1YW0L9,26415\nBRF6FX9,90819\nBZBFKT7,31499\nB0D0006,113669\nB2RLCR0,12713\n6440859,26523\nBD0YVN2,13580\nBLGM738,21084\nB01RM25,744743\n6322173,409997\nB12LR51,197907\n6624471,484421\n6606996,1108139\nB59L4L7,824340\nBC9ZH86,64449\nB5B1TX2,1509800\n6280215,175778\nB03N0C7,290585\nB1FFT76,228434\n6673570,178451\nBN4N9C0,53688\n7243530,55410\n5263251,56314\nB03MVJ8,174104\n4519449,44816\n6503138,196659\nBD5ZXH8,63789\nB9GFHQ6,86875\nBYXW419,30465\n6563206,199835\nBN0VX82,27107\nBG087C6,75367\n6889526,1109683\n6628008,45833\nB59SNS8,72519\nB59HPK1,107150\n7320154,46369\nBNC0DB0,280371\nB03NGS5,141250\nB63DG21,147630\n6148197,795213\nB12LZW4,112592\nB3C8VY3,197368\nBJTM270,360592\nB12LSY7,121258\nB1324D0,173192\nBMZQ749,33425\nB12M7Q5,287416\n4767981,1374068\n6801575,100821\nB128FN6,7160\nBSHYYN1,374393\nB03MYN3,124340\nBNGCVY1,57257

In [4]:
import sys
import os

# Add the sibling directory to Python's search path
sibling_dir = os.path.abspath("../utils")
sys.path.append(sibling_dir)

from openfigi_utils import read_api_key, query_openfigi_by_sedol

api_key = read_api_key("/home/craigc/.keys/openfigi")

In [5]:
figi_data = {}

In [6]:
df = pd.read_csv("sedols_and_quantities.txt")

for sedol in df['SEDOL']:

    if sedol in figi_data:
        continue
    
    sedol_result = query_openfigi_by_sedol(api_key, sedol)

    if 'error' in sedol_result[0]:
        # print('failed to find SEDOL:', sedol)
        print(sedol)

    figi_data[sedol] = sedol_result

    # print(sedol_result)

In [7]:
securities_db_cols = [
    "sedol",
    "isin",
    "figi",
    "name",
    "ticker",
    "exchange",
    "type",
    "sector"
]

securities_db = pd.DataFrame(columns=securities_db_cols)

for sedol in df['SEDOL']:
    if sedol not in figi_data:
        print(sedol)
        break

    if sedol in securities_db["sedol"].values:
        continue

    data = figi_data[sedol][0]['data'][0]

    # display(data)

    # Define a row as a dictionary
    new_row = {
        "sedol": sedol,
        "isin": "", # TODO
        "figi": data['figi'],
        "name": data['name'],
        "ticker": data['ticker'],
        "exchange": data['exchCode'],
        "type": data['securityType'],
        "sector": data['marketSector']
    }

    securities_db = pd.concat([securities_db, pd.DataFrame([new_row])], ignore_index=True)

  
display(securities_db)
    

,sedol,isin,figi,name,ticker,exchange,type,sector
0,6545464,,BBG000DKRVC9,ABU DHABI COMMERCIAL BANK,ADCB,UH,Common Stock,Equity
1,BN12D39,,BBG00M6JJSK9,ADNOC DRILLING CO PJSC,ADNOCDRI,UH,Common Stock,Equity
2,B03MN70,,BBG000BV9XY9,AKBANK T.A.S.,AKBNK,TI,Common Stock,Equity
3,B403QG4,,BBG000M294S2,MOUWASAT MEDICAL SERVICES CO,MOUWASAT,AB,Common Stock,Equity
4,B12LZH9,,BBG000FBK1L0,AL RAJHI BANK,RJHI,AB,Common Stock,Equity
...,...,...,...,...,...,...,...,...
59,B128FN6,,BBG000QS6Y35,CO FOR COOPERATIVE INSURANCE,TAWUNIYA,AB,Common Stock,Equity
60,BSHYYN1,,BBG0069P8N84,THE SAUDI NATIONAL BANK,SNB,AB,Common Stock,Equity
61,B03MYN3,,BBG000F101G0,TURKCELL ILETISIM HIZMET AS,TCELL,TI,Common Stock,Equity
62,BNGCVY1,,BBG008F2T3T2,UNITED CO RUSAL INTERNATIONA,RUAL,RM,Common Stock,Equity


In [8]:
securities_db.to_csv("../data/securities_db.csv", index=False)

In [16]:
import time

holdings_db_cols = [
    "sedol",
    "quantity",
    "report_date",
    "created_at",
    "updated_at",
    "deleted_at",
]

holdings_db = pd.DataFrame(columns=holdings_db_cols)

for _, row in df.iterrows():
    sedol = row['SEDOL']
    quantity = row['Quantity']

    if sedol not in figi_data:
        print(sedol)
        break

    if sedol in holdings_db["sedol"].values:
        continue

    data = figi_data[sedol][0]['data'][0]

    timestamp = int(time.time())

    # Define a row as a dictionary
    new_row = {
        "sedol": sedol,
        "quantity": quantity,
        "report_date": val_date,
        "created_at": timestamp,
        "updated_at": timestamp,
    }

    holdings_db = pd.concat([holdings_db, pd.DataFrame([new_row])], ignore_index=True)
  
display(holdings_db)

AttributeError: 'Series' object has no attribute 'Sedol'

In [14]:
holdings_db.to_csv("bemo_holdings.csv", index=False)